# <font color='red'> Description du projet </font>

## <font color='blue'>Présentation du problème </font>

L’objectif de ce projet est d’estimer **les temps de réponse et de mobilisation** de la Brigade des Pompiers de Londres. La brigade des pompiers de Londres est le service d'incendie et de sauvetage le plus actif du Royaume-Uni  et l'une des plus grandes organisations de lutte contre l'incendie et de sauvetage au monde.

Le premier jeu de données fourni contient les détails de chaque incident traité depuis janvier 2009. Des informations sont fournies sur la date et le lieu de l'incident ainsi que sur le type d'incident traité. Il est composé de deux fichiers

*   LFB Incident data from 2009 - 2017.xlsx
*   LFB Incident data from 2018 onwards.csv

Le second fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-incident-records/f5066d66-c7a3-415f-9629-026fbda61822/LFB%20Incident%20data%20from%202018%20onwards.csv.xlsx' pour avoir la dernière version du fichier. En effet, les données sont mises à jour tous les mois. Il faut compter au moins 7 minutes pour la lecture des données.

<br>

Le second jeu de données contient les détails de chaque camion de pompiers envoyé sur les lieux d'un incident depuis janvier 2009. Des informations sont fournies sur l'appareil mobilisé, son lieu de déploiement et les heures d'arrivée sur les lieux de l'incident. Il est composé de trois fichiers

*   LFB Mobilisation data from January 2009 - 2014.xlsx
*   LFB Mobilisation data from 2015 - 2020.xlsx
*   LFB Mobilisation data from January 2009 - 2014.xlsx

Le dernier fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-mobilisation-records/3ff29fb5-3935-41b2-89f1-38571059237e/LFB%20Mobilisation%20data%202021%20-%202024.xlsx' pour avoir la dernière version du fichier (mise à jour mensuelle). Il faut compter environ 17 minutes pour la lecture des données.

## <font color='blue'> Etapes précédentes </font>

*   1 - Exploration des données : premières analyses, concaténation des différents fichiers puis jointure des 2 types de données (incident / mobilisation)
*   2 - Data visualisation.ipynb : visualisation des données, étude de la variable à prédire (temps de réponse total) en fonction des variables explicatives, création d'un jeu de données pour la modélisation
*   5 - Preprocessing final : création des jeux de données pour la modélisation faite dans ce notebook
*   6 - Modelisation Preproc final - Pred Continue : modélisation de la variable continue `TotalResponseTime_BC`



*Nota Bene* : Les notebooks numérotés 3 et 4 ont été conservés pour montrer un premier travail de modélisation qui a été abandonné par la suite (voir détails ci-dessous)


## <font color='blue'>Etapes dans ce notebook </font>

Dans ce notebook, nous avons modélisé le temps de réponse, après catégorisation en 10 classes (`ResponseTimeCategory`). Nous avons testé plusieurs algorithmes de classification.

Pour évaluer nos modèles (et les comparer) nous utilisons les métriques suivantes :
- l'exactitude (accuracy) calculée sur la totalité des classes
- la précision médiane, minimale et maximale (cette mesure étant calculée pour chaque classe)
- le rappel (recall) médian, minimal et maximal
- le f1-score médian, minimal et médian
Ces mesures sont calculées sur le jeu de données de validation.

Les résultats étant mauvais - accuracy de l'ordre de 25% à 28% pour tous les modèles - nous avons peu commenté ce notebook.

# <font color='red'>1) Préparation de l'environement de travail </font>

## <font color='blue'>Installation des modules </font>

In [ ]:
#!pip install matplotlib
#!pip install Seaborn
#!pip install openpyxl
#!pip install scipy
#!pip install geopandas
#!pip install scikit-learn
#!pip install statsmodels
#!pip install folium
#!pip install plotly
#!pip install --upgrade seaborn
#!pip install jupyter
#!pip install nbformat
!pip install lightgbm
!pip install xgboost
#!pip optuna

## <font color='blue'>Importation des bibliothèques </font>

In [ ]:
import pandas as pd  #Pour les dataframe
import numpy as np #Pour le calcul numérique
import datetime as dt # Pour le calcul sur les dates

## Libraries pour les graphiques
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D # graphique 3D
import plotly.express as px  #graphique 3D dynamique
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as offline

## Libraries pour statistiques
# régression linéaire
import statsmodels.api as sm
from statsmodels.formula.api import ols
# tests statistiques
from scipy.stats import shapiro , kstest # tests de sur la normalité de la distributin
from scipy.stats import bartlett # tests sur les variances
from scipy.stats import kruskal #  comparaison des médianes
from scipy.stats import spearmanr
from scipy.stats import loguniform, uniform
from scipy import stats # notamment pour boxplot
# metrics
from sklearn.metrics import f1_score, accuracy_score, make_scorer, classification_report, recall_score, precision_recall_fscore_support

# Libraries divers
from copy import deepcopy  # gestion des copies

# Pour la séparation du jeu de données
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, LogisticRegression

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor, XGBClassifier
# import optuna


## <font color='blue'>Liaison avec le drive (pour travailler sur GoogleColab) </font>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# <font color='red'>2) Données pour la modélisation </font>

## <font color='blue'>2.a) Chargement </font>


Chargement des données depuis GoogleColab

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset.csv', low_memory=False)
validation = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset.csv', low_memory=False)

train_reduit = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset.csv', low_memory=False)
validation_reduit = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset.csv', low_memory=False)

Chargement des données depuis un emplacement en local

In [ ]:
train = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset.csv', low_memory=False)
validation = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset.csv', low_memory=False)

train_reduit = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset.csv', low_memory=False)
validation_reduit = pd.read_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset.csv', low_memory=False)

## <font color='blue'>2.b) Sélection des variables </font>

In [ ]:
# X : variable explicative du modele --> on supprime IncidentNumber et les 4 variables à prédire
# y : la variable à prédire est ResponseTimeCategory
X_train = train.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train = train[['ResponseTimeCategory']]

X_val = validation.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val = validation[['ResponseTimeCategory']]


# même principe sur le jeux de données réduit (pour la recherche d'hyperparamètres)
X_train_r = train_reduit.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train_r = train_reduit[['ResponseTimeCategory']]

X_val_r = validation_reduit.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val_r = validation_reduit[['ResponseTimeCategory']]


# <font color='red'> 3) Méthodes
 </font>







Dans la suite de ce notebook, nous utilisons les méthodes définies ici

In [ ]:
### evaluation du modèle
# cette méthode calcul les différentes métriques permettant l'évaluation du modèle
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name}")
    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred)
    print("Overall accuracy :", np.round(accuracy_score(y_true, y_pred),4))
    print("min precision: ", np.round(precision.min(),4))
    print("median precision : ", np.round(np.median(precision),4))
    print("max precision : ", np.round(precision.max(),4))
    print("min recall : ", np.round(recall.min(),4))
    print("median recall : ", np.round(np.median(recall),4))
    print("max recall : ", np.round(recall.max(),4))
    print("min fscore : ", np.round(fscore.min(),4))
    print("median fscore : ", np.round(np.median(fscore),4))
    print("max fscore : ", np.round(fscore.max(),4))


In [ ]:
### estimation des paramètres, prédiction et calcul des métriques d'évaluation
def train_and_evaluate_model(model, model_name, X_train, y_train, X_val, y_val):
    y_train = y_train.to_numpy().ravel() if isinstance(y_train, pd.DataFrame) else y_train
    y_val = y_val.to_numpy().ravel() if isinstance(y_val, pd.DataFrame) else y_val
    # Entraînement du modèle
    model.fit(X_train, y_train)

    # Prédiction sur l'ensemble de validation
    y_pred_val = model.predict(X_val)

    # Évaluation sur l'ensemble de validation
    evaluate_model(y_val, y_pred_val, model_name)

    return y_pred_val

In [ ]:
def find_best_params_and_predict(model, param_distributions, X_train, y_train, X_val, y_val, model_name, n_iter):
    """
    Effectue une recherche des meilleurs paramètres pour un modèle donné avec RandomizedSearchCV, et retourne les prédictions.
    """
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,               # Nombre d'itérations de recherche aléatoire
        scoring='accuracy',      # métrique à utiliser pour le choix d'hyper paramètres
        cv=3,                    # Nombre de sub dataset pour la validation croisée
        n_jobs=-1,               # Utilisation de tous les cœurs disponibles
        verbose=1,               # Affiche les logs de progression
        random_state=42          # Pour garantir la reproductibilité
    )

    # Exécute la recherche
    randomized_search.fit(X_train, y_train)

    # Meilleurs paramètres et score
    print(f"\n{model_name} - Best Parameters: {randomized_search.best_params_}")
    print(f"{model_name} - Best Cross-Validation Score: {randomized_search.best_score_:.4f}")

    # Meilleur modèle entraîné
    best_model = randomized_search.best_estimator_

    # Prédictions sur l'ensemble de validation
    y_pred = best_model.predict(X_val)

    # Évaluation
    evaluate_model(y_val, y_pred, model_name)

    return y_pred, randomized_search.best_params_


# <font color='red'> 4) Classification
 </font>


## <font color='blue'>4.a) Support Vector Machine (SVM) </font>

Nous n'avons pas réussi à faire tourner les modèles SVM que ce soit sur le jeux de données complet ou réduit. Ce type de modèles a donc été abandonné

In [ ]:
svm_model = SVC(random_state=42)
y_pred_svm = train_and_evaluate_model(svm_model, "SVM", X_train, y_train, X_val, y_val)

## <font color='blue'>4.b) K-plus proches voisins </font>

### Première estimation

In [ ]:
knn_model = KNeighborsClassifier()
y_pred_KNC=train_and_evaluate_model(knn_model, "k-Nearest Neighbors", X_train, y_train, X_val, y_val)


k-Nearest Neighbors
Overall accuracy : 0.2453
min precision:  0.1506
median precision :  0.1926
max precision :  0.4589
min recall :  0.1005
median recall :  0.2005
max recall :  0.5772
min fscore :  0.1206
median fscore :  0.2049
max fscore :  0.4575


### Recherche des hyper paramètres

In [ ]:
param_distributions_knn = {
    'n_neighbors': list(range(1, 31)),   # Nombre de voisins à considérer
    'weights': ['uniform', 'distance'],  # Pondération des voisins
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Métriques de distance
}

In [ ]:
knn_model = KNeighborsClassifier()

In [ ]:
y_pred_best_KNN, best_params_KNN = find_best_params_and_predict(
    model=knn_model,
    param_distributions=param_distributions_knn,
    X_train=X_train_r,
    y_train=y_train_r_ravel,
    X_val=X_val_r,
    y_val=y_val_r_ravel,
    model_name="KNN",
    n_iter=20  # Nombre d'itérations pour la recherche aléatoire
)


Fitting 3 folds for each of 20 candidates, totalling 60 fits

KNN - Best Parameters: {'weights': 'uniform', 'n_neighbors': 30, 'metric': 'manhattan'}
KNN - Best Cross-Validation Score: 0.2508

KNN
Overall accuracy : 0.2554
min precision:  0.1357
median precision :  0.1967
max precision :  0.4413
min recall :  0.0507
median recall :  0.198
max recall :  0.5542
min fscore :  0.0739
median fscore :  0.1979
max fscore :  0.4914


## <font color='blue'>4.c) Arbre de décision </font>

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
y_pred_DT= train_and_evaluate_model(dt_model, "Decision Tree", X_train, y_train, X_val, y_val)


Decision Tree
Overall accuracy : 0.2352
min precision:  0.1351
median precision :  0.2014
max precision :  0.4119
min recall :  0.1204
median recall :  0.1994
max recall :  0.4896
min fscore :  0.1273
median fscore :  0.1986
max fscore :  0.4474


In [ ]:
feat_imp = pd.DataFrame({'importance': dt_model.feature_importances_}, index=dt_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.715792
ratioStd,0.045969
H1117,0.030514
Bor_inc_rep,0.020841
PropCat_Non Residential,0.014784
PropCat_Dwelling,0.013352
PropCat_Road Vehicle,0.012303
PropCat_Outdoor Structure,0.011927
PropCat_Outdoor,0.008147
H26,0.007675


### Recherche des hyper paramètres

In [ ]:
param_distributions_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 10],
    'max_features': ['sqrt', 'log2', None]
}

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

In [ ]:
y_train_r_ravel = y_train_r.values.ravel()  # Convertit en NumPy puis aplati
y_val_r_ravel = y_val_r.values.ravel()

In [ ]:
y_pred_best_DT, best_params_DT = find_best_params_and_predict(
    model=dt_model,
    param_distributions=param_distributions_dt,
    X_train=X_train_r,
    y_train=y_train_r_ravel,
    X_val=X_val_r,
    y_val=y_val_r_ravel,
    model_name="DecisionTree",
    n_iter=20  # Nombre d'itérations pour la recherche aléatoire
)

Fitting 3 folds for each of 20 candidates, totalling 60 fits

DecisionTree - Best Parameters: {'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10, 'criterion': 'entropy'}
DecisionTree - Best Cross-Validation Score: 0.2664

DecisionTree
Overall accuracy : 0.2689
min precision:  0.1402
median precision :  0.2086
max precision :  0.4945
min recall :  0.0232
median recall :  0.1982
max recall :  0.5558
min fscore :  0.0414
median fscore :  0.2074
max fscore :  0.5191


## <font color='blue'>4.d) Forêt aléatoire </font>

### Première estimation

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
y_pred_RFC= train_and_evaluate_model(rf_model, "Random Forest", X_train, y_train, X_val, y_val)
# y_pred_RFC = rf_model.predict(X_val)
# evaluate_model(y_val, y_pred_RFC, "Random Forest")


Random Forest
Overall accuracy : 0.2373
min precision:  0.1331
median precision :  0.203
max precision :  0.4448
min recall :  0.1265
median recall :  0.2032
max recall :  0.4669
min fscore :  0.1297
median fscore :  0.2031
max fscore :  0.4556


In [ ]:
feat_imp = pd.DataFrame({'importance': rf_model.feature_importances_}, index=rf_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.915426
Stat_resp_rep,0.012966
ratioStd,0.009235
Bor_inc_rep,0.007271
H1117,0.006282
Bor_resp_rep,0.005819
H26,0.005508
PropCat_Non Residential,0.002759
PropCat_Dwelling,0.002627
PropCat_Road Vehicle,0.002374


### Recherche des hyper paramètres

In [ ]:
param_distributions_rf = {
    'n_estimators': [50, 100, 200, 500],       # Nombre d'arbres
    'max_depth': [None, 10, 20, 30],           # Profondeur maximale (None = pas de limite)
    'min_samples_split': [2, 5, 10],           # Nombre min d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4],             # Nombre min d'échantillons dans une feuille
    'max_features': ['sqrt', 'log2', None],    # Nombre de features considérés pour chaque split
    'bootstrap': [True, False]                 # Utilisation ou non du bootstrap
}



In [ ]:
rf_model = RandomForestClassifier(random_state=42)
y_pred_best_RFC, best_params_RF = find_best_params_and_predict(
    model=rf_model,
    param_distributions=param_distributions_rf,
    X_train=X_train_r,
    y_train=y_train_r,
    X_val=X_val_r,
    y_val=y_val_r,
    model_name="RandomForest Classification",
    n_iter=20  # Nombre d'itérations (augmenter pour une meilleure optimisation)
)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




RandomForest Classification - Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 10, 'bootstrap': True}
RandomForest Classification - Best Cross-Validation Score: 0.2757

RandomForest Classification
Overall accuracy : 0.2744
min precision:  0.1731
median precision :  0.2118
max precision :  0.4808
min recall :  0.0079
median recall :  0.23
max recall :  0.5822
min fscore :  0.0151
median fscore :  0.2126
max fscore :  0.5267


## <font color='blue'>4.e) Gradient Boosting </font>

In [ ]:
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
y_pred_GBC= train_and_evaluate_model(gbm_model, "Gradient Boosting", X_train, y_train, X_val, y_val)



Gradient Boosting
Overall accuracy : 0.2757
min precision:  0.1744
median precision :  0.2153
max precision :  0.4659
min recall :  0.0045
median recall :  0.2296
max recall :  0.6072
min fscore :  0.0087
median fscore :  0.2178
max fscore :  0.5272


### Recherche des hyper paramètres

In [ ]:
param_distributions_gbc = {
    'n_estimators': [50, 100, 200, 500],       # Nombre d'arbres dans le modèle
    'learning_rate': [0.01, 0.05, 0.1, 0.2],   # Taux d'apprentissage
    'max_depth': [3, 5, 7, 10],                # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],           # Nombre minimum d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4],             # Nombre minimum d'échantillons dans une feuille
    'subsample': [0.6, 0.8, 1.0],              # Proportion d’échantillons utilisés pour chaque arbre
    'max_features': ['sqrt', 'log2', None]     # Nombre de features considérés pour chaque split
}

In [ ]:
gbc_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

In [ ]:
y_pred_best_GBC, best_params_GBC = find_best_params_and_predict(
    model=gbc_model,
    param_distributions=param_distributions_gbc,
    X_train=X_train_r,
    y_train=y_train_r_ravel,
    X_val=X_val_r,
    y_val=y_val_r_ravel,
    model_name="GradientBoosting",
    n_iter=20  # Nombre d'itérations (peut être augmenté pour une meilleure optimisation)
)


Fitting 3 folds for each of 20 candidates, totalling 60 fits

GradientBoosting - Best Parameters: {'subsample': 1.0, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 5, 'learning_rate': 0.1}
GradientBoosting - Best Cross-Validation Score: 0.2733

GradientBoosting
Overall accuracy : 0.2746
min precision:  0.1076
median precision :  0.2159
max precision :  0.4837
min recall :  0.0118
median recall :  0.219
max recall :  0.5843
min fscore :  0.0213
median fscore :  0.21
max fscore :  0.5261


## <font color='blue'>4.f) XG Boost </font>

In [ ]:
from xgboost import XGBClassifier
model_name = "XGBoost Classification"
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGBC= train_and_evaluate_model(xgb_model, model_name, X_train, y_train, X_val, y_val)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[08:12:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGBoost Classification
Overall accuracy : 0.2747
min precision:  0.1797
median precision :  0.2156
max precision :  0.4524
min recall :  0.0003
median recall :  0.2339
max recall :  0.6217
min fscore :  0.0005
median fscore :  0.217
max fscore :  0.5237


### Recherche des hyper paramètres

In [ ]:
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0, 0.01, 0.1, 1]
}


In [ ]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Gọi hàm để tìm kiếm siêu tham số tốt nhất và dự đoán
y_pred_XGBC, best_params_XGBC = find_best_params_and_predict(
    model=xgb_model,
    param_distributions=param_distributions,
    X_train=X_train_r,
    y_train=y_train_r,
    X_val=X_val_r,
    y_val=y_val_r,
    model_name="XGBoost Classification",
    n_iter=20
)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[08:47:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGBoost Classification - Best Parameters: {'subsample': 1.0, 'reg_lambda': 0.01, 'reg_alpha': 1, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.8}
XGBoost Classification - Best Cross-Validation Score: 0.2751

XGBoost Classification
Overall accuracy : 0.2734
min precision:  0.1615
median precision :  0.2067
max precision :  0.4725
min recall :  0.0114
median recall :  0.2235
max recall :  0.599
min fscore :  0.0213
median fscore :  0.2111
max fscore :  0.5229


## <font color='blue'>4.g) LGBM </font>

In [ ]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
y_pred_LGBMC= train_and_evaluate_model(lgbm_model, "LightGBM", X_train, y_train, X_val, y_val)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:


Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.




[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095348 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 383
[LightGBM] [Info] Number of data points in the train set: 726399, number of used features: 50
[LightGBM] [Info] Start training from score -2.331608
[LightGBM] [Info] Start training from score -2.571446
[LightGBM] [Info] Start training from score -2.289115
[LightGBM] [Info] Start training from score -2.155095
[LightGBM] [Info] Start training from score -2.135044
[LightGBM] [Info] Start training from score -2.225498
[LightGBM] [Info] Start training from score -2.389665
[LightGBM] [Info] Start training from score -2.644152
[LightGBM] [Info] Start training from score -2.345349
[LightGBM] [Info] Start training from score -2.084298

LightGBM
Overall accuracy 

### Recherche des hyper paramètres

In [ ]:
param_distributions_lgbm = {
    'n_estimators': [50, 100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [-1, 3, 5, 10],
    'num_leaves': [20, 31, 40, 50],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0, 0.01, 0.1, 1]
}

In [ ]:
lgbm_model = LGBMClassifier(random_state=42)

y_pred_LGBM, best_params_LGBM = find_best_params_and_predict(
    model=lgbm_model,
    param_distributions=param_distributions_lgbm,
    X_train=X_train_r,
    y_train=y_train_r,
    X_val=X_val_r,
    y_val=y_val_r,
    model_name="LGBM",
    n_iter=20
)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020649 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 384
[LightGBM] [Info] Number of data points in the train set: 139989, number of used features: 50
[LightGBM] [Info] Start training from score -2.326726
[LightGBM] [Info] Start training from score -2.594388
[LightGBM] [Info] Start training from score -2.317475
[LightGBM] [Info] Start training from score -2.182694
[LightGBM] [Info] Start training from score -2.146663
[LightGBM] [Info] Start training from score -2.228858
[LightGBM] [Info] Start training from score -2.383484
[LightGBM] [Info] Start training from score -2.616728
[LightGBM] [Info] Start training from score -2.317403
[LightGBM] [Info] Start training from score -2.054865
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

# <font color='red'> 6) Conclusions
 </font>


Les résultats de la modélisation de la variable temps catégorisée en 10 classes ne sont pas satisfaisants (accuracy de 25% environ). Dans le prochain notebook, nous modélisons le temps catégorisé en 6 classes. Nous espérons une amélioration des mesures dû à la simplification de la variable à expliquer.